In [7]:
from pyspark.sql import SparkSession
from hr_metadata_service import HRMetadataService
from workforce_nlp_processor import WorkforceNLPProcessor
from hr_query_engine import execute_plan
from datetime import date, timedelta

spark = SparkSession.builder.appName("workforce-idqs-notebook").getOrCreate()
meta = HRMetadataService(data_dir="data").load()
nlp = WorkforceNLPProcessor()

def run(plan, title):
    print(f"\n=== {title} ===")
    res = execute_plan(spark, meta, plan)
    print("rows:", res.get("count"))
    if res.get("rows"): print(res["rows"][:5])
    return res

# Headcount by division
plan1 = {"intent":"organizational_analysis","tables":["workforce"],
         "select":["division"],"group_by":["division"],
         "aggregations":[{"function":"count_distinct","column":"personIdExternal","alias":"headcount"}],
         "order_by":[{"column":"headcount","direction":"desc"}],"limit":1000,"glossary_refs":["headcount"],
         "business_context":"Headcount grouped by division"}
run(plan1, "Headcount by division")

# Hired last N months with active assignments
since = (date.today() - timedelta(days=30*6)).isoformat()
plan2 = {"intent":"cross_table_analysis","tables":["workforce","assignment"],
         "joins":[{"left_table":"workforce","right_table":"assignment","on":"personIdExternal"}],
         "where":[{"table":"workforce","column":"startDate","operator":">=","value":since},
                  {"table":"assignment","column":"assignmentStatus","operator":"=","value":"A"}],
         "select":["personIdExternal","startDate","division","department","assignmentType","assignmentStatus"],
         "order_by":[{"column":"startDate","direction":"desc"}],"limit":20,
         "glossary_refs":["active_assignment"],
         "business_context":f"Hired since {since} with active assignments"}
run(plan2, "Hired last 6 months with active assignments")

# Average tenure by department, full-time employees
plan3 = {"intent":"workforce_analytics","tables":["workforce","assignment"],
         "joins":[{"left_table":"workforce","right_table":"assignment","on":"personIdExternal"}],
         "where":[{"table":"assignment","column":"assignmentType","operator":"in","value":["FULL-TIME","FULL TIME","FT"]},
                  {"table":"workforce","column":"status","operator":"=","value":"A"}],
         "select":["department"],"group_by":["department"],
         "aggregations":[{"function":"avg_tenure_years","column":"startDate","alias":"avg_tenure_years"}],
         "order_by":[{"column":"avg_tenure_years","direction":"desc"}],"limit":1000,
         "glossary_refs":["tenure_years","full_time","active_employees"],
         "business_context":"Average tenure by department for full-time active employees"}
run(plan3, "Average tenure by department (full-time)")

# Managers with >5 reports
plan4 = {"intent":"management_hierarchy","tables":["workforce"],
         "select":["managerId","department"],"group_by":["managerId","department"],
         "aggregations":[{"function":"count","column":"personIdExternal","alias":"direct_reports"}],
         "having":[{"column":"direct_reports","operator":">","value":5}],
         "order_by":[{"column":"direct_reports","direction":"desc"}],"limit":1000,
         "business_context":"Managers with >5 direct reports by department"}
run(plan4, "Managers with >5 direct reports")

# NLP example
qry = "Show headcount by division and employment status"
plan5 = nlp.parse(qry)
run(plan5, f"NLP: {qry}")



=== Headcount by division ===
rows: 6
[{'division': 'Marketing', 'headcount': 76}, {'division': 'Operations', 'headcount': 72}, {'division': 'HR', 'headcount': 67}, {'division': 'Finance', 'headcount': 67}, {'division': 'Sales', 'headcount': 59}]

=== Hired last 6 months with active assignments ===
rows: 20
[{'personIdExternal': 'EMP00087', 'startDate': '2025-08-04', 'division': 'Sales', 'department': 'Enterprise Sales', 'assignmentType': 'PART-TIME', 'assignmentStatus': 'A'}, {'personIdExternal': 'EMP00131', 'startDate': '2025-08-04', 'division': 'Operations', 'department': 'Supply Chain', 'assignmentType': 'PART-TIME', 'assignmentStatus': 'A'}, {'personIdExternal': 'EMP00087', 'startDate': '2025-08-04', 'division': 'Sales', 'department': 'Enterprise Sales', 'assignmentType': 'CONTRACT', 'assignmentStatus': 'A'}, {'personIdExternal': 'EMP00055', 'startDate': '2025-07-25', 'division': 'Operations', 'department': 'Supply Chain', 'assignmentType': 'CONTRACT', 'assignmentStatus': 'A'}, {

IndexError: list index out of range

In [ ]:
from importlib import reload
import hr_metadata_service
reload(hr_metadata_service)
from hr_metadata_service import HRMetadataService

meta = HRMetadataService(data_dir="data").load()
print("Glossary keys:", sorted(meta["business_terms"].keys())[:10], "…")


In [ ]:
from importlib import reload
import hr_query_engine
reload(hr_query_engine)
from hr_query_engine import execute_plan


In [ ]:
res = execute_plan(spark, meta, plan2)  # the "Hired last 6 months with active assignments" plan
len(res["rows"]), res["rows"][:3]
